In [30]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
import numpy as np 
import skimage
from skimage import transform
from skimage import io 
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
#import keras

In [5]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


INFO:plaidml:Opening device "metal_amd_radeon_r9_m370x.0"


In [6]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
x_train[0:32].shape

(32, 32, 32, 3)

In [8]:
model.fit(x_train[0:32], y_train[0:32])

Epoch 1/1


INFO:plaidml:Analyzing Ops: 133 of 320 operations complete


32/32 [==============================] - 2s 65ms/step - loss: 2.2604 - acc: 0.1562


In [9]:
%%time 
epochs = 1
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
49984/50000 [============================>.] - ETA: 0s - loss: 1.8259 - acc: 0.3361

INFO:plaidml:Analyzing Ops: 127 of 320 operations complete


50000/50000 [==============================] - 142s 3ms/step - loss: 1.8258 - acc: 0.3362 - val_loss: 1.5660 - val_acc: 0.4315
CPU times: user 25.6 s, sys: 21.3 s, total: 46.9 s
Wall time: 2min 22s
